# FIle Structure and Vraiable Definition
- `origin_data`: From DB 的原始資料
- `origin_data_csv`: 原始資料轉檔為 `.csv`
- `logs`: 實驗輸出檔案資料夾
- `Z:` : 網路磁碟機(WFDB NAS)
- `alive_yuran` : alive_42731_withHRV.csv, pd.Dataframe
- `dead_yuran` : dead_42731_withHRV.csv, pd.Dataframe
- `alive_set` : alive_yuran 的 subject_id 集合, set
- `dead_set` : dead_yuran 的 subject_id 集合, set
- `total_set` : `alive_set` and `dead_set` 的聯集, set

In [1]:
ORIGIN_DATA = "origin_data"
DATA_CSV = "origin_data_csv"
LOGS = "logs"
MATCH = "Z:"
import pandas as pd

import numpy as np
import wfdb                                      # 讀取 WFDB header / record :contentReference[oaicite:4]{index=4}
from pathlib import Path                         # 物件導向檔案操作 :contentReference[oaicite:5]{index=5}
from datetime import datetime, timedelta, date, time
from tqdm import tqdm                            # 進度列（可省略）
import logging, os                               # 紀錄檔與系統路徑
from collections import defaultdict
import logging
import os
import ast
import re
from typing import List, Optional, Tuple,Set
from  tqdm import tqdm

def cross_validation_missing_subject(fileA: str | pd.DataFrame, fileB: str | pd.DataFrame, fileAname:str, fileBname:str):
    """
    列出
    - 哪些 subject id 存在於 fileA 但不存在於 fileB
    - 哪些 subject id 存在於 fileB 但不存在於 fileA
    並將結果列出

    Arg:
    - fileA: file path of fileA
    - fileB: file path of fileB
    """
    if not isinstance(fileA,pd.DataFrame):
        fileA_df = pd.read_csv(fileA)
    else:
        fileA_df = fileA

    if not isinstance(fileB,pd.DataFrame):
        fileB_df = pd.read_csv(fileB)
    else:
        fileB_df = fileB
        

    # 確認欄位名稱（假設欄位叫 SUBJECT_ID）
    if 'SUBJECT_ID' not in fileA_df.columns or 'SUBJECT_ID' not in fileB_df.columns:
        raise ValueError("Both files must contain 'SUBJECT_ID' column")

    # 轉換為集合
    setA = set(fileA_df['SUBJECT_ID'].dropna().astype(str))
    setB = set(fileB_df['SUBJECT_ID'].dropna().astype(str))

    # 找差集
    only_in_A = setA - setB
    only_in_B = setB - setA

    print(f"✅ SUBJECT_ID 存在於 {fileAname} 但不存在於 {fileBname}, 共 {len(only_in_A)}:")
    print(only_in_A if only_in_A else "無")
    
    print(f"\n✅ SUBJECT_ID 存在於 {fileBname} 但不存在於 {fileAname}共 {len(only_in_B)}:")
    print(only_in_B if only_in_B else "無")

    # 回傳結果（以 dict）
    return {
        "only_in_A": only_in_A,
        "only_in_B": only_in_B
    }

try:
    alive_yuran = pd.read_csv("./experiment_data_from_yuran/alive_42731_withHRV.csv")
    dead_yuran = pd.read_csv("./experiment_data_from_yuran/dead_42731_withHRV.csv")

    alive_set = set(alive_yuran['SUBJECT_ID'].to_list())
    dead_set = set(dead_yuran["SUBJECT_ID"].to_list())

    total_set = alive_set | dead_set
except Exception as e:
    print(e)



In [2]:
# ================================= 初始化紀錄 =================================

if os.path.isdir(LOGS):
    print(f"{LOGS} folder exist.")
else:
    os.makedirs("logs", exist_ok=True)
    print(f"{LOGS} folder doesn't exist, creating new {LOGS}")

# 建立 Logger
try:
    logger = logging.getLogger("data_clean")
    logger.setLevel(logging.WARNING)  # WARNING 以上都會被記錄

    # 建立 FileHandler，寫入 logs/clean.log
    fh = logging.FileHandler(f"{LOGS}/clean.log", mode="w", encoding="utf-8")
    # 只輸出訊息本身：SUBJECT_ID REASON
    formatter = logging.Formatter("%(message)s")
    fh.setFormatter(formatter)

    # 避免重複加入 handler
    if not logger.handlers:
        logger.addHandler(fh)
    print("Logger module create success.")
except Exception as e:
    raise ValueError(e)

def record_log(subject_id: str, reason: str):
    """
    將不合格的資料記錄到 logs/clean.log。
    例如： logger.warning("12345 invalid_date")
    """
    logger.warning(f"{subject_id} {reason}")

logs folder exist.
Logger module create success.


In [3]:
# ================================= 篩選基本 Clinical Data: Output: base.csv =================================
def load_clinical_tables()->pd.DataFrame:
    """
    篩選 具備 ICD Code 42731診斷碼與ICU紀錄的病患，***並且只保留該病患最後一筆ICUSTAY紀錄***
    Return:
    - base : pd.DataFrame, 患者綜合表格，包含以下資訊
        來自 patients.csv
        - SUBJECT_ID：患者唯一識別碼。

        admissions 表格
        - HADM_ID：住院請求編號，用於唯一識別一次住院。
        - ADMITTIME：病患入院時間（時間戳記）。
        - DISCHTIME：病患出院時間（時間戳記）。
        - HOSPITAL_EXPIRE_FLAG：出院時是否死亡（1 = 出院時已逝，0 = 否）。
        - DEATHTIME: 院內死亡時間

        icustays 表格
        - ICUSTAY_ID：ICU 住院期間的唯一識別符。
        - INTIME：進入 ICU 的時間戳記。
        - OUTTIME：離開 ICU 的時間戳記。

        diagnoses_icd 表格
        - ICD9_CODE：使用 ICD‑9 編碼系統記錄的診斷代碼（最多 6 位字元，包含空格，有些是 V 開頭代碼）
    """
    adm   = pd.read_csv(os.path.join(DATA_CSV,"ADMISSIONS.csv"), usecols=['SUBJECT_ID','HADM_ID',
                                                        'ADMITTIME','DISCHTIME','DEATHTIME',
                                                        'HOSPITAL_EXPIRE_FLAG'])

    icu   = pd.read_csv(os.path.join(DATA_CSV,"ICUSTAYS.csv"),
                        usecols=['SUBJECT_ID','ICUSTAY_ID','HADM_ID','INTIME','OUTTIME'])

    diag  = pd.read_csv(os.path.join(DATA_CSV,"DIAGNOSES_ICD.csv"),
                        usecols=['SUBJECT_ID','HADM_ID','ICD9_CODE'])
    # ICD9 42731 = AF
    # ── 1. 找出 AF 病人 ───────────────────────
    af_subjects = diag.loc[diag['ICD9_CODE']=='42731', 'SUBJECT_ID'].unique()

    # ── 2. 保留這些病人的全部 ICU stay ───────
    icu_af = icu[icu['SUBJECT_ID'].isin(af_subjects)]

    # ── 3. 加入 ADMISSIONS 資料 ───────────────
    base = icu_af.merge(adm, on=['SUBJECT_ID', 'HADM_ID'], how='left')

    # ── 4. 標記該次住院是否含 AF ──────
    af_flag = (diag[diag['ICD9_CODE']=='42731']
            [['SUBJECT_ID','HADM_ID']]
            .drop_duplicates()
            .assign(HAS_AF=1))
    base = base.merge(af_flag,
                    on=['SUBJECT_ID','HADM_ID'],
                    how='left') \
            .fillna({'HAS_AF': 0})

    # ── 5. 去重（若 diag 同 HADM_ID 多筆 42731）──
    base = base.drop_duplicates(subset=['ICUSTAY_ID'])

    # 刪除非 AF 患者
    base = base[base['HAS_AF'] == 1]

    # (New) 只保留最後一筆 ICU 紀錄
    print("只取每個病人最後一筆 ICU紀錄")
    # 先依 SUBJECT_ID 升序、INTIME 降序排序
    base = base.sort_values(by=['SUBJECT_ID', 'INTIME'], ascending=[True, False])
    # 對每個病人取第一筆（也就是最後一筆 ICU）
    base = base.drop_duplicates(subset='SUBJECT_ID', keep='first')

    # 儲存
    base.to_csv(os.path.join(LOGS, "base.csv"), index=False)
    return base

base = load_clinical_tables()

print("總資料: ",len(base))
print("總共多少 subject id",base['SUBJECT_ID'].nunique())

"""Analysis with Final Set"""
print(f"Missing ID (total {len(total_set - set(base['SUBJECT_ID'].to_list()))}): {total_set - set(base['SUBJECT_ID'].to_list())}")





只取每個病人最後一筆 ICU紀錄
總資料:  10252
總共多少 subject id 10252
Missing ID (total 0): set()


In [4]:
# ================================= 根據 base.csv 結果篩選 WFDB ECG 訊號: 該 Module 移送至 ecg_cache.py->Output: ecg_match_info.csv  =================================
# base.csv: 保證 subject id 不重複
# ecg_match_info.csv: 單一subject id 會對應到多筆 header
#此階段 Output: ecg_match_info.csv


def match_base_ecg(base:pd.DataFrame, ecg_match_info:pd.DataFrame)->pd.DataFrame:
    """
    針對每一筆 base（住院資料）依據 SUBJECT_ID，在 ecg_match_info（ECG 檔案對應表）中搜尋是否有時間區間重疊的 ECG 訊號資料。
    若同時有多筆重疊，僅擷取 T1_lead2 最晚結束時間那筆；若只有一筆則直接擷取；若無符合則填 None。
    最終回傳一份含有原 base 欄位與所有對應 ECG 資訊的新 DataFrame，並將結果存檔為 base_match_ecg_df.csv。

    Args:
    - base: pd.DataFrame  
        住院資料，需包含 'SUBJECT_ID', 'INTIME', 'OUTTIME' 等欄位，作為比對主體。
    - ecg_match_info: pd.DataFrame  
        ECG 檔案對應資料表，需包含 'SUBJECT_ID', 'T0', 'T1', 'T1_lead2', 'PREFIX', 'FOLDER', 'HEADER', 'Total_lead2_sec' 等欄位。

    Return:
    - base_match_ecg_df: pd.DataFrame  
        新增 ECG 資訊的 base 資料，每筆 row 增加 ECG 欄位（無符合者為 None），並自動儲存結果 csv，回傳完整 DataFrame。
    """
    for col in ["T0","T1","T1_lead2"]:
        ecg_match_info[col] = pd.to_datetime(ecg_match_info[col],errors="raise")

    for col in ["INTIME","OUTTIME"]:
        base[col] = pd.to_datetime(base[col],errors="raise")


    PREFIX = []
    FOLDER= []
    HEADER= []
    T0= []
    T1= []
    T1_lead2= []
    Total_lead2_sec= []
    match_count = 0
    for _,row in base.iterrows():
        ecgs = ecg_match_info[ecg_match_info['SUBJECT_ID']==row['SUBJECT_ID']] # May have many ecg records
        intime = row['INTIME']
        outtime = row['OUTTIME']
        overlaps = (
            (ecgs['T0'].clip(lower=intime) <
            ecgs['T1_lead2'].clip(upper=outtime))
        )
        overlaps_match = overlaps[overlaps==True]

        if len(overlaps_match)>1:
            # 多筆 ECG Header Match: 擷取最後一段
            multple_ecg_headers = ecg_match_info.iloc[overlaps_match.index]
            latest_row = multple_ecg_headers.sort_values(by='T1_lead2', ascending=False).iloc[0]
            PREFIX.append(latest_row['PREFIX'])
            FOLDER.append(latest_row['FOLDER'])
            HEADER.append(latest_row['HEADER'])
            T0.append(latest_row['T0'])
            T1.append(latest_row['T1'])
            T1_lead2.append(latest_row['T1_lead2'])
            Total_lead2_sec.append(latest_row['Total_lead2_sec'])
            match_count+=1
        elif len(overlaps_match) == 1:
            idx = overlaps_match.index[0]                # 抓第一個 index（scalar）
            latest_row = ecg_match_info.iloc[idx]        # 現在才會回傳 Series
            PREFIX.append(latest_row['PREFIX'])
            FOLDER.append(latest_row['FOLDER'])
            HEADER.append(latest_row['HEADER'])
            T0.append(latest_row['T0'])
            T1.append(latest_row['T1'])
            T1_lead2.append(latest_row['T1_lead2'])
            Total_lead2_sec.append(latest_row['Total_lead2_sec'])
            match_count+=1
        else:
            PREFIX.append(None)
            FOLDER.append(None)
            HEADER.append(None)
            T0.append(None)
            T1.append(None)
            T1_lead2.append(None)
            Total_lead2_sec.append(None)


    base_match_ecg_df = base.copy()
    base_match_ecg_df['PREFIX'] = PREFIX
    base_match_ecg_df['FOLDER'] = FOLDER
    base_match_ecg_df['HEADER'] = HEADER
    base_match_ecg_df['T0'] = T0
    base_match_ecg_df['T1'] = T1
    base_match_ecg_df['T1_lead2'] = T1_lead2
    base_match_ecg_df['Total_lead2_sec'] = Total_lead2_sec

    base_match_ecg_df.to_csv(os.path.join(LOGS,"base_match_ecg_df.csv"),index=False)
    print(f"Total Records {match_count} matched!")
    return base_match_ecg_df
ecg_match_info = pd.read_csv(os.path.join(LOGS,"ecg_match_info.csv"))
base_match_ecg_df=match_base_ecg(base,ecg_match_info)

#刪除沒有 match的 row 方便作分析
base_match_ecg_df = base_match_ecg_df.dropna(subset=['T0','T1','T1_lead2'], axis=0)


Total Records 2237 matched!


In [5]:

base_match_ecg_df.info()
print(f"Missing : {total_set - set(base_match_ecg_df['SUBJECT_ID'].to_list())}")

filtered_df = base_match_ecg_df[
    base_match_ecg_df['SUBJECT_ID'].isin(total_set) &
    (base_match_ecg_df['Total_lead2_sec'] < 36000)
]

print(f"\nException Patient(Match LEADII < 36000):\n{filtered_df['SUBJECT_ID']}")

<class 'pandas.core.frame.DataFrame'>
Index: 2237 entries, 25 to 15755
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   SUBJECT_ID            2237 non-null   int64         
 1   HADM_ID               2237 non-null   int64         
 2   ICUSTAY_ID            2237 non-null   int64         
 3   INTIME                2237 non-null   datetime64[ns]
 4   OUTTIME               2237 non-null   datetime64[ns]
 5   ADMITTIME             2237 non-null   object        
 6   DISCHTIME             2237 non-null   object        
 7   DEATHTIME             403 non-null    object        
 8   HOSPITAL_EXPIRE_FLAG  2237 non-null   int64         
 9   HAS_AF                2237 non-null   float64       
 10  PREFIX                2237 non-null   object        
 11  FOLDER                2237 non-null   object        
 12  HEADER                2237 non-null   object        
 13  T0                   

In [6]:
# base_match_ecg_df 時間分析: 分析 T0/T1_lead2 的時間段，與'ADMITTIME','DISCHTIME','DEATHTIME','INTIME', 'OUTTIME'等欄位的交叉狀況與異常數值分析
# 考量記錄誤差，可以加入時間誤差容忍 : tolerance_min, tolerance
# 目前預設 0 (以輸出為準)
# 避免紀錄誤差刪去太多紀錄：時間誤差容忍　30 mins


for col in ['ADMITTIME','DISCHTIME','DEATHTIME','INTIME', 'OUTTIME', 'T0', 'T1','T1_lead2']:
    base_match_ecg_df[col] = pd.to_datetime(base_match_ecg_df[col], errors='coerce')
print(len(base_match_ecg_df.dropna(subset=['ADMITTIME','DISCHTIME','INTIME', 'OUTTIME', 'T0', 'T1','T1_lead2'])))

# 1. ECG: T0/T1_Lead2 是否落在 ICU INTIME/OUTTIME
tolerance_min = 0
tolerance = pd.Timedelta(minutes=tolerance_min)
print(f"=============== Current Error Tolerance : {tolerance_min} min ===============")
in_ICU_OUTTIME = (
    # (base_match_ecg_df['T0'] >= (base_match_ecg_df['INTIME'] - tolerance)) &
    (base_match_ecg_df['T1_lead2'] <= (base_match_ecg_df['OUTTIME'] + tolerance))
)

in_ICU_INTIME = (
    (base_match_ecg_df['T0'] >= (base_match_ecg_df['INTIME'] - tolerance))
)

ecg_in_ICU_OUT = base_match_ecg_df[in_ICU_OUTTIME]
ecg_in_ICU_OUT_set = set(ecg_in_ICU_OUT['SUBJECT_ID'].to_list())
ecg_in_ICU_IN = base_match_ecg_df[in_ICU_INTIME]
ecg_in_ICU_IN_set = set(ecg_in_ICU_IN['SUBJECT_ID'].to_list())
print(f"Missing (Out of ICU OUTTIME)(len : {len(total_set - ecg_in_ICU_OUT_set)}): {sorted(list(total_set - ecg_in_ICU_OUT_set))}")
print(f"Missing (Before  ICU INTIME)(len : {len(total_set - ecg_in_ICU_IN_set)}): {sorted(list(total_set - ecg_in_ICU_IN_set))}")

# 2. ECG: T0/T1_Lead2 是否落在 ADMISSIONS ADMITTIME/DISCHTIME
in_ADM_ADMITTIME = (
    (base_match_ecg_df['T0'] >= (base_match_ecg_df['ADMITTIME'] - tolerance))
)

in_ADM_DISCHTIME = (
    (base_match_ecg_df['T1_lead2'] <= (base_match_ecg_df['DISCHTIME'] + tolerance))
)

ecg_in_ADM_OUT = base_match_ecg_df[in_ADM_DISCHTIME]
ecg_in_ADM_OUT_set = set(ecg_in_ADM_OUT['SUBJECT_ID'].to_list())
ecg_in_ADM_IN = base_match_ecg_df[in_ADM_ADMITTIME]
ecg_in_ADM_IN_set = set(ecg_in_ADM_IN['SUBJECT_ID'].to_list())

print(f"Missing (Out of ADM DISCHTIME)(len : {len(total_set - ecg_in_ADM_OUT_set)}): {sorted(list(total_set - ecg_in_ADM_OUT_set))}")
print(f"Missing (Before  ADM ADMITTIME)(len : {len(total_set - ecg_in_ADM_IN_set)}): {sorted(list(total_set - ecg_in_ADM_IN_set))}")

# 3 Outlier Analysis: 
# 3.1 所有 ICU INTIME 都小於 ADM　ADMITTIME (30min tolerance)
intime_smaller_than_admittime = (
    (base_match_ecg_df['INTIME']+tolerance >= (base_match_ecg_df['ADMITTIME']))
)
outlier_intime_earlier_than_admittime = base_match_ecg_df[~intime_smaller_than_admittime]
outlier_intime_earlier_than_admittime_set = set(outlier_intime_earlier_than_admittime['SUBJECT_ID'].to_list())
print(f"Outliter - ICU INTIME earlier than ADMIITIME ({len(outlier_intime_earlier_than_admittime)}): {outlier_intime_earlier_than_admittime['SUBJECT_ID'].to_list()}")
print(f"Outlier intersect with total set ({len(total_set & outlier_intime_earlier_than_admittime_set)}): {sorted(list(total_set & outlier_intime_earlier_than_admittime_set)) }")

#3.2 所有 ICU OUTTIME 都早於 ADM DISCHTIME (30min tolerance)
outtime_smaller_than_dischtime = (
    (base_match_ecg_df['T1_lead2'] <= (base_match_ecg_df['DISCHTIME'] + tolerance))
)
outlier_outtime_later_than_dischtime = base_match_ecg_df[~outtime_smaller_than_dischtime]
outlier_outtime_later_than_dischtime_set = set(outlier_outtime_later_than_dischtime['SUBJECT_ID'].to_list())
print(f"Outliter - ICU OUTTIME later than DISCHTIME ({len(outlier_outtime_later_than_dischtime_set)}): {outlier_outtime_later_than_dischtime_set}")
print(f"Outlier intersect with total set ({len(total_set & outlier_outtime_later_than_dischtime_set)}): {sorted(list(total_set & outlier_outtime_later_than_dischtime_set)) }")


2237
=============== Current Error Tolerance : 0 min ===============
Missing (Out of ICU OUTTIME)(len : 22): [5685, 14197, 19866, 26055, 26732, 27428, 27884, 30170, 48777, 51277, 55402, 56751, 64944, 65055, 69144, 72282, 75476, 81475, 85866, 86968, 88521, 89091]
Missing (Before  ICU INTIME)(len : 31): [1354, 2172, 10013, 14266, 16258, 16463, 17667, 19718, 22337, 43400, 44437, 45608, 45942, 47543, 48011, 50762, 54209, 60659, 68607, 73760, 75886, 75998, 79709, 80254, 83598, 84142, 84845, 84874, 92777, 93031, 93898]
Missing (Out of ADM DISCHTIME)(len : 27): [6605, 7251, 9258, 14266, 16463, 24626, 26271, 28180, 30047, 32012, 43084, 43673, 51277, 55725, 57299, 57968, 72048, 74913, 75883, 77689, 81475, 81593, 84914, 85979, 88503, 94351, 99067]
Missing (Before  ADM ADMITTIME)(len : 0): []
Outliter - ICU INTIME earlier than ADMIITIME (44): [2514, 10320, 10342, 11591, 15569, 18572, 32067, 41902, 42093, 42721, 43060, 45806, 46802, 50141, 52802, 54147, 54429, 55616, 60074, 60852, 64374, 64384, 71

In [7]:
# 劃分存活組 Surv, 死亡組 Mort, 尚未過濾沒有 Output 
def split_surv_and_mort(base_match_ecg_df:pd.DataFrame)->Tuple[pd.DataFrame,pd.DataFrame]:
    """
    依據 ICU 住院紀錄與 ECG 訊號時間範圍，將資料依存活（survival）與死亡（mortality）組進行嚴格分組。
    分組與過濾條件如下：

    - **ECG 覆蓋條件（兩組皆需同時滿足，允許 30 分鐘誤差 tolerance）：**
        設 ICU 期間為 [INTIME, OUTTIME]，ECG 覆蓋期間為 [T0, T1_lead2]
        - (1) T0 >= INTIME - tolerance
        - (2) T1_lead2 <= OUTTIME + tolerance

    - **死亡組（mort）定義：**
        - HOSPITAL_EXPIRE_FLAG == 1
        - DEATHTIME 不為空（即已知死亡時間）
        - ECG 覆蓋條件成立（如上）
        - ECG 訊號長度 Total_lead2_sec >= 36,000（即總共有10小時長度的訊號，可以不連續）

        公式：
        ```
        mort = base_match_ecg_df[
            (T0 >= INTIME - tolerance) &
            (T1_lead2 <= OUTTIME + tolerance) &
            (HOSPITAL_EXPIRE_FLAG == 1) &
            (DEATHTIME 非空) &
            (Total_lead2_sec >= 36000)
        ]
        ```

    - **存活組（surv）定義：**
        - HOSPITAL_EXPIRE_FLAG == 0
        - DEATHTIME 為空（即未於住院期間死亡）
        - ECG 覆蓋條件成立（如上）
        - ECG 訊號長度 Total_lead2_sec >= 36,000（即連續10小時）

        公式：
        ```
        surv = base_match_ecg_df[
            (T0 >= INTIME - tolerance) &
            (T1_lead2 <= OUTTIME + tolerance) &
            (HOSPITAL_EXPIRE_FLAG == 0) &
            (DEATHTIME 為空) &
            (Total_lead2_sec >= 36000)
        ]
        ```

    Args:
    - base_match_ecg_df: pd.DataFrame  
        已合併 ICU 與 ECG 資訊的 DataFrame，需包含 ICU 入出時間、ECG 開始結束時間、死亡標記等必要欄位。

    Return:
    - surv: pd.DataFrame  
        存活組資料表（完全滿足上述存活條件）。
    - mort: pd.DataFrame  
        死亡組資料表（完全滿足上述死亡條件）。

    篩選過程中會自動轉換時間欄位型別、去除不齊全資料，並輸出分組人數與 SUBJECT_ID 是否有重疊。
    """
   
    for col in ['ADMITTIME','DISCHTIME','DEATHTIME','INTIME', 'OUTTIME', 'T0', 'T1','T1_lead2']:
        base_match_ecg_df[col] = pd.to_datetime(base_match_ecg_df[col], errors='coerce')
        
    base_match_ecg_df.dropna(subset=['ADMITTIME','DISCHTIME','INTIME', 'OUTTIME', 'T0', 'T1','T1_lead2'],inplace=True)
    # 1. ECG: T0/T1_Lead2 是否落在 ICU INTIME/OUTTIME (with 30min 誤差容忍)
    tolerance_min = 0
    tolerance = pd.Timedelta(minutes=tolerance_min)
    print(f"=============== Current Error Tolerance : {tolerance_min} min ===============")

    in_ICU_OUTTIME = (
        # (base_match_ecg_df['T0'] >= (base_match_ecg_df['INTIME'] - tolerance)) &
        (base_match_ecg_df['T1_lead2'] <= (base_match_ecg_df['OUTTIME'] + tolerance))
    )

    in_ICU_INTIME = (
        (base_match_ecg_df['T0'] >= (base_match_ecg_df['INTIME'] - tolerance))
    )

    #split group
    mort = base_match_ecg_df[
        (in_ICU_OUTTIME) &
        (in_ICU_INTIME) &
        (base_match_ecg_df['HOSPITAL_EXPIRE_FLAG'] == 1) &
        (~base_match_ecg_df['DEATHTIME'].isnull()) &
        (base_match_ecg_df['Total_lead2_sec'] >= 36000)
    ]

    surv = base_match_ecg_df[
        (in_ICU_OUTTIME) &
        (in_ICU_INTIME) &
        (base_match_ecg_df['HOSPITAL_EXPIRE_FLAG']==0) &
        (base_match_ecg_df['DEATHTIME'].isnull()) &
        (base_match_ecg_df['Total_lead2_sec'] >= 36000)
    ]

    #分析有無重疊
    print(f"Total Surv: {len(surv)}\nTotal Mort:{len(mort)}")
    print(f"重疊 subject id : {set(mort['SUBJECT_ID'].to_list()) & set(surv['SUBJECT_ID'].to_list())}")

    return surv, mort

surv, mort=split_surv_and_mort(base_match_ecg_df)

=============== Current Error Tolerance : 0 min ===============
Total Surv: 1384
Total Mort:285
重疊 subject id : set()


In [8]:
def merge_dob_and_calculate_age(df:pd.DataFrame) -> pd.DataFrame:
    """
    將輸入資料（df）根據 SUBJECT_ID 合併 PATIENTS.csv 內的 DOB 欄位，並計算各病患的入院時年齡（AGE_YEARS），
    以及標記是否小於 125 歲（AGE_UNDER_125）。同時檢查資料合理性（ADMITTIME 必須晚於 DOB），
    並於過程中列印無效日期的樣本資料供檢查。

    處理步驟與產出：
    1. 合併 df 與 PATIENTS.csv（以 SUBJECT_ID 對應），並強制轉換日期格式。
    2. 計算 ADMITTIME 與 DOB 差的年數，存為 AGE_YEARS 欄位。
    3. 標記年齡是否小於 125 歲（AGE_UNDER_125，布林值）。
    4. 列印 ADMITTIME <= DOB 的異常樣本前 5 筆供檢查。
    5. 回傳已新增年齡資訊之完整 DataFrame。

    Args:
    - df: pd.DataFrame  
        欲補齊出生日期與計算年齡的主資料表，必須包含 'SUBJECT_ID' 及 'ADMITTIME' 欄位。

    Return:
    - df_with_Age: pd.DataFrame  
        回傳合併後的資料表，新增欄位：
        - 'DOB': 病患出生日期
        - 'AGE_YEARS': 入院時年齡（以年計）
        - 'AGE_UNDER_125': 是否小於 125 歲（布林值）
    """
    try:
        pat = pd.read_csv(os.path.join(DATA_CSV, "PATIENTS.csv"),
                          usecols=['SUBJECT_ID', 'DOB'])
    except FileNotFoundError:
        raise ValueError(f"Required CSV files not found in {DATA_CSV}")

    pat['DOB'] = pd.to_datetime(pat['DOB'], errors='coerce')
    df['ADMITTIME'] = pd.to_datetime(df['ADMITTIME'], errors='coerce')

    print(f"Before Merge, Total id : {df['SUBJECT_ID'].nunique()}")
    df_with_Age = df.merge(pat, on="SUBJECT_ID", how="left")
    df_with_Age = df_with_Age.reset_index(drop=True)
    print(f"After Merge, Total id : {df_with_Age['SUBJECT_ID'].nunique()}")
    

    # 加入檢查 & 過濾
    mask_valid = (
        (df_with_Age['ADMITTIME'] > df_with_Age['DOB'])
    )
    print("Invalid date rows:")
    print(df_with_Age[~mask_valid][['SUBJECT_ID', 'DOB', 'ADMITTIME']].head(5))

    df_with_Age['AGE_YEARS'] = df_with_Age['ADMITTIME'].dt.year - df_with_Age['DOB'].dt.year
    df_with_Age['AGE_UNDER_125'] = (df_with_Age['AGE_YEARS'] < 125)



    return df_with_Age


In [9]:
# 死亡組分析
# Output: Stage 2 最終結果，detailed 保留所有資訊 , filtered 過濾掉不合格資料，僅保留全部符合的 data
# 1. mort_stage2_detailed.csv
# 2. mort_stage2_filtered.csv
mort_with_age = merge_dob_and_calculate_age(mort)

# Post ECG :T1_lead2 出現在死亡後或出現後
print(f"容忍誤差: {tolerance}")
post_ecg_icu = (
    ((mort_with_age['DEATHTIME']+tolerance )< mort_with_age['T1_lead2'])
)

post_ecg_dischtime = (
    ((mort_with_age['DEATHTIME']+tolerance) < mort_with_age['DISCHTIME'])
)

mort_with_post_ecg = mort_with_age[(post_ecg_icu | post_ecg_dischtime)]
mort_with_post_ecg_set = set(mort_with_post_ecg['SUBJECT_ID'].to_list())

print(f"Total Mort Post ECG({len(mort_with_post_ecg_set)}) : {mort_with_post_ecg_set}")
print(f"dead_42731 with post ecg ({len(dead_set & mort_with_post_ecg_set)}):{dead_set & mort_with_post_ecg_set}")

mort_with_age['post_ecg_icu'] = post_ecg_icu
mort_with_age['post_ecg_dischtime'] = post_ecg_dischtime
mort_with_age.to_csv(os.path.join(LOGS,"mort_stage2_detailed.csv"),index=False)

# 過濾: Age under 125 and without post ecg(採用嚴格定義：deathtime 必須晚於 T1_lead2結束或離院時間(取小，且 ICU OUTTIME 已經在 Split 時檢查過這邊不再檢查))

mort_final = mort_with_age[
    (mort_with_age['AGE_UNDER_125']) &
    (~(post_ecg_icu | post_ecg_dischtime))
]
mort_final.to_csv(os.path.join(LOGS,"mort_stage2_filtered.csv"),index=False)
mort_final_set = set(mort_final['SUBJECT_ID'].to_list())

print(f"Final Mort : {len(mort_final)}")
print(f"Missing({len(dead_set-mort_final_set)}) : {sorted(list(dead_set-mort_final_set))}")
print(f"Addtion({len(mort_final_set-dead_set)}) : {sorted(list(mort_final_set-dead_set))}")

Before Merge, Total id : 285
After Merge, Total id : 285
Invalid date rows:
Empty DataFrame
Columns: [SUBJECT_ID, DOB, ADMITTIME]
Index: []
容忍誤差: 0 days 00:00:00
Total Mort Post ECG(50) : {98182, 32012, 30477, 2063, 21775, 94351, 28180, 75670, 43673, 69146, 19102, 26271, 28073, 9258, 99115, 55725, 18738, 24626, 84914, 88503, 10552, 81593, 25915, 92475, 9289, 43084, 23503, 46927, 66770, 57299, 95957, 85979, 90846, 53856, 95201, 81378, 21219, 61667, 74727, 80744, 60393, 65513, 75883, 73068, 57968, 72048, 74866, 42995, 77689, 99067}
dead_42731 with post ecg (19):{32012, 94351, 28180, 43673, 26271, 9258, 55725, 24626, 84914, 88503, 81593, 43084, 57299, 85979, 75883, 57968, 72048, 77689, 99067}
Final Mort : 212
Missing(34) : [2172, 9258, 10013, 14266, 16258, 16463, 24626, 26271, 28180, 30047, 32012, 43084, 43673, 48011, 51277, 55725, 57299, 57968, 60659, 72048, 74913, 75883, 77689, 79709, 81475, 81593, 84142, 84845, 84914, 85979, 88503, 93031, 94351, 99067]
Addtion(142) : [124, 593, 743, 10

C:\Users\louislin\AppData\Local\Temp\ipykernel_14288\18883387.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ADMITTIME'] = pd.to_datetime(df['ADMITTIME'], errors='coerce')


In [10]:
#存活組分析
# Output: Stage 2 最終結果，detailed 保留所有資訊 , filtered 過濾掉不合格資料，僅保留全部符合的 data
# 1. surv_stage2_detailed.csv
# 2. surv_stage2_filtered.csv
surv_with_age = merge_dob_and_calculate_age(surv)

# Post Discharge ECG signal : T1_lead2 >= DISCHTIME or OUTTIME
surv_post_ecg_icu = (
    surv_with_age['T1_lead2'] >= (surv_with_age['OUTTIME'] + tolerance)
)

surv_post_ecg_dischtime = (
    surv_with_age['T1_lead2'] >= (surv_with_age['DISCHTIME'] + tolerance)
)

surv_early_ecg_icu = (
    surv_with_age['T0'] <= (surv_with_age['INTIME'] - tolerance)
)

surv_early_ecg_admittime = (
    surv_with_age['T0'] <= (surv_with_age['ADMITTIME'] - tolerance)
)

surv_with_post_ecg = surv_with_age[(surv_post_ecg_icu | surv_post_ecg_dischtime)]
surv_with_post_ecg_set = set(surv_with_post_ecg['SUBJECT_ID'].to_list())
surv_with_post_ecg.to_csv(os.path.join(LOGS,"test.csv"),index=False)
print(f"Total Surv Post ECG({len(surv_with_post_ecg)}) : {surv_with_post_ecg_set}")
print(f"alive_42731 with post ecg ({len(alive_set & surv_with_post_ecg_set)}):{alive_set & surv_with_post_ecg_set}")

surv_with_post_ecg['post_ecg_icu'] = surv_post_ecg_icu
surv_with_post_ecg['post_ecg_dischtime'] = surv_post_ecg_dischtime

surv_with_post_ecg.to_csv(os.path.join(LOGS,"surv_stage2_detailed.csv"),index=False)

# 過濾: Age under 125 and without post ecg(採用嚴格定義：T1_lead2 必須早於 ICU OUTTIME或離院時間(取小))，並且不會有住院前或進ICU前 ICU 紀錄

surv_final = surv_with_age[
    (surv_with_age['AGE_UNDER_125']) &
    (~((surv_post_ecg_icu | surv_post_ecg_dischtime))) &
    (~((surv_early_ecg_icu | surv_early_ecg_admittime)))
]
surv_final.to_csv(os.path.join(LOGS,"surv_stage2_filtered.csv"),index=False)
surv_final_set = set(surv_final['SUBJECT_ID'].to_list())

print(f"Final Surv : {len(surv_final)}")
print(f"Missing({len(alive_set - surv_final_set)}) : {sorted(list(alive_set - surv_final_set))}")
print(f"Addtion({len(surv_final_set-alive_set)}) : {sorted(list(surv_final_set-alive_set))}")


Before Merge, Total id : 1384
After Merge, Total id : 1384
Invalid date rows:
Empty DataFrame
Columns: [SUBJECT_ID, DOB, ADMITTIME]
Index: []
Total Surv Post ECG(17) : {70273, 4802, 76801, 74503, 61928, 9993, 45801, 78410, 83751, 6605, 54736, 50385, 7251, 16499, 94164, 11512, 24825}
alive_42731 with post ecg (2):{7251, 6605}
Final Surv : 1284
Missing(42) : [1354, 5685, 6605, 7251, 14197, 17667, 19718, 19866, 22337, 26055, 26732, 27428, 27884, 30170, 43400, 44437, 45608, 45942, 47543, 48777, 50762, 54209, 55402, 56751, 64944, 65055, 68607, 69144, 72282, 73760, 75476, 75886, 75998, 80254, 83598, 84874, 85866, 86968, 88521, 89091, 92777, 93898]
Addtion(960) : [214, 217, 262, 625, 695, 708, 735, 849, 1012, 1072, 1144, 1313, 1418, 1449, 1528, 1563, 1900, 1941, 1949, 2157, 2395, 2442, 2467, 2514, 2549, 2747, 2754, 2981, 3214, 3351, 3365, 3512, 3515, 3695, 3744, 3768, 3886, 4018, 4059, 4180, 4266, 4420, 4477, 4788, 4807, 5114, 5126, 5307, 5343, 5606, 6039, 6063, 6070, 6090, 6229, 6335, 6561, 

C:\Users\louislin\AppData\Local\Temp\ipykernel_14288\18883387.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ADMITTIME'] = pd.to_datetime(df['ADMITTIME'], errors='coerce')
C:\Users\louislin\AppData\Local\Temp\ipykernel_14288\2362036729.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surv_with_post_ecg['post_ecg_icu'] = surv_post_ecg_icu
C:\Users\louislin\AppData\Local\Temp\ipykernel_14288\2362036729.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [ ]:
# Surv Addtion Analysis : 這邊以後是跟原檔案(42731)交叉對比，不用管
cols = ['SUBJECT_ID','HADM_ID','ICUSTAY_ID','INTIME','OUTTIME','ADMITTIME','DISCHTIME','T0','T1_lead2','Total_lead2_sec','HEADER']
surv_final_addition = surv_final[surv_final['SUBJECT_ID'].isin(surv_final_set-alive_set)][cols]
surv_final_addition.to_csv(os.path.join(LOGS,"surv_stage2_addition.csv"),index=False)


In [24]:
# Surv: [subject_id, hadm_id] 嚴格交叉比對
cols = ['SUBJECT_ID', 'HADM_ID']
surv_final_same = surv_final[surv_final['SUBJECT_ID'].isin(alive_set & surv_final_set)]

set_df2 = set(zip(surv_final_same['SUBJECT_ID'], surv_final_same['HADM_ID']))

alive_yuran['in_df2'] = list(zip(alive_yuran['SUBJECT_ID'], alive_yuran['HADM_ID']))
alive_yuran['in_df2'] = alive_yuran['in_df2'].apply(lambda x: x in set_df2)

print(alive_yuran[['SUBJECT_ID', 'HADM_ID', 'in_df2']])

all_in = alive_yuran['in_df2'].all()
print("全部都有對應" if all_in else "有缺漏")

not_in_df2 = alive_yuran[~alive_yuran['in_df2']]
print("以下組合沒在df2出現：")
print(not_in_df2[['SUBJECT_ID','HADM_ID']])





     SUBJECT_ID  HADM_ID  in_df2
0         12372   176291    True
1          1409   124337    True
2         14186   171902    True
3         15218   159946    True
4         16353   152248    True
..          ...      ...     ...
361       94811   177259    True
362       94853   197339    True
363       95313   185643    True
364        9537   128113    True
365       96148   135552    True

[366 rows x 3 columns]
有缺漏
以下組合沒在df2出現：
     SUBJECT_ID  HADM_ID
7         19866   168234
13        26055   196340
14        27884   128997
19        30170   120612
27        43400   124673
40        47543   184250
86        65055   153824
95        68607   124149
99        69144   153453
110       73760   141240
112       75886   175305
132       83598   107203
137       84874   101728
141       86968   191498
147       88521   166573
170        1354   144830
174       14197   170569
182       17667   197869
185       19718   178054
187       22337   125014
195       26732   125708
198       274

In [25]:
# Mort: [subject_id, hadm_id] 嚴格交叉比對
cols = ['SUBJECT_ID', 'HADM_ID']
mort_final_same = mort_final[mort_final['SUBJECT_ID'].isin(dead_set & mort_final_set)]

set_df2 = set(zip(mort_final_same['SUBJECT_ID'], mort_final_same['HADM_ID']))

dead_yuran['in_df2'] = list(zip(dead_yuran['SUBJECT_ID'], dead_yuran['HADM_ID']))
dead_yuran['in_df2'] = dead_yuran['in_df2'].apply(lambda x: x in set_df2)

print(dead_yuran[['SUBJECT_ID', 'HADM_ID', 'in_df2']])

all_in = dead_yuran['in_df2'].all()
print("全部都有對應" if all_in else "有缺漏")

not_in_df2 = dead_yuran[~dead_yuran['in_df2']]
print("以下組合沒在df2出現：")
print(not_in_df2[['SUBJECT_ID','HADM_ID']])
# Missing in Mort: [2172, 24626, 30047, 48011, 51277, 74913, 81475, 84142]

     SUBJECT_ID  HADM_ID  in_df2
0          1049   117138    True
1         10629   193631    True
2         16463   183203   False
3         16924   193729    True
4         18875   189045    True
..          ...      ...     ...
99         9258   183354   False
100       94351   166930   False
101       96746   140835    True
102       97801   165472    True
103       99067   130940   False

[104 rows x 3 columns]
有缺漏
以下組合沒在df2出現：
     SUBJECT_ID  HADM_ID
2         16463   183203
7         26271   191492
10        30047   172127
13        51277   103045
16        57968   179996
21        81475   101662
22        81593   192553
24        84142   133986
26        84914   198880
27        88503   100784
29        93031   125979
30        10013   165520
33        14266   111581
37        16258   148188
42         2172   181252
43        24626   167669
47        28180   119762
52        32012   138125
55        43084   169551
57        43673   121255
60        48011   166293
64        557